In [5]:
!pip install selenium
!pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/ce/cd/a7c2cbffe2afff975349e60b14b63a448162145a7acac8ba12ddc2ed78a8/pandas-2.1.1-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for numpy>=1.22.4 from https://files.pythonhosted.org/packages/cc/05/ef9fc04adda45d537619ea956bc33489f50a46badc949c4280d8309185ec/numpy-1.26.0-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.1 kB ? eta -:--:--
     ------------------------- ------------ 41.0/61.1 kB 991.0 kB/s eta 0:00:01
     ---------------------------------------- 61.1/61.1 kB 1.1 MB/s eta 0:00:00
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   -----

In [2]:
# Libraries
import time
import pandas as pd    
# ------------- # 
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains #to scrolldown 
import math
import re
import time

start_time = time.strftime("%H:%M:%S", time.localtime())
print("Starting_Time: ",start_time)

def starting_page():
    # Driver path
    driver = webdriver.Chrome()  

    # Maximize Window
    driver.maximize_window() 
    driver.minimize_window()  
    driver.maximize_window()  
    driver.switch_to.window(driver.current_window_handle)
    driver.implicitly_wait(10)

    # Enter to the site
    driver.get('https://www.linkedin.com/login');
    time.sleep(2)

    # Accept cookies

    # User Credentials
    email = "email_name@outlook.com"
    password = "password"
    position = "data engineer intern"
    local = "india"

    driver.find_element(By.XPATH,'//*[@id="username"]').send_keys(email)
    driver.find_element(By.XPATH,'//*[@id="password"]').send_keys(password)
    time.sleep(1)

    # Login button
    driver.find_element(By.XPATH,"//button[@aria-label='Sign in']").click()
    driver.implicitly_wait(10)

    # Jobs page
    driver.get('https://www.linkedin.com/jobs/search')
    ## waiting load
    time.sleep(2)

    # Go to search results directly
    # driver.get("https://www.linkedin.com/jobs/search/?geoId=105646813&keywords=junior%20data%20analyst&location=Chennai")
    driver.find_element(By.XPATH,'//*[@aria-label="Search by title, skill, or company"]').send_keys(position)
    driver.find_element(By.XPATH,'//*[@aria-label="City, state, or zip code"]').clear()
    driver.find_element(By.XPATH,'//*[@aria-label="City, state, or zip code"]').send_keys(local)
    driver.find_element(By.CLASS_NAME,'jobs-search-box__submit-button').click()
    time.sleep(30)

    #date posted buttons - past week
    date_posted_xpath = "//button[@aria-label='Date posted filter. Clicking this button displays all Date posted filter options.']"
    driver.find_element(By.XPATH,date_posted_xpath).click()
    time.sleep(30)
    driver.find_element(By.XPATH,"//span[text()='Past week']").click()
    #show results button - multiple show results buttons present - taking the 1st search button for now
    list_date_modified = driver.find_elements(By.CSS_SELECTOR,'div.reusable-search-filters-buttons>button:nth-child(2)>span')
    list_date_modified[0].click()
    time.sleep(10)

    #all buttons - jobs, date posted, experience level, company, on-site remote
    all_buttions =driver.find_elements(By.CSS_SELECTOR,"button.artdeco-pill:nth-child(1)")
    #took all the buttons and selected the 3rd one for experience level

    #experience level
    all_buttions[2].click() #clicking the experience level button
    time.sleep(20)
    driver.find_element(By.XPATH,"//span[text()='Internship']").click()
    time.sleep(3)
    driver.find_element(By.XPATH,"//span[text()='Entry level']").click()
    time.sleep(3)
    #show results button - multiple show results buttons present - taking the 2nd search button for now
    list_date_modified = driver.find_elements(By.CSS_SELECTOR,'div.reusable-search-filters-buttons>button:nth-child(2)>span')
    list_date_modified[1].click()
    time.sleep(10)

    #total results to calculate the no of pages
    total_results_text = driver.find_element(By.CSS_SELECTOR,'div.jobs-search-results-list__subtitle').text
    print(total_results_text)
    total_results= int("".join(re.findall(r'\d+', total_results_text)))
    total_pages = math.ceil(int(total_results)/25)
    print(total_pages)
    # Get all links for these offers

    return driver, total_pages

functioncall = starting_page()
driver = functioncall[0]
total_pages = functioncall[1]

# Navigate 13 pages
print('Links are being collected now.')
links = []
job_titles_list = []
company_names_list = []
location_names_list = []
posted_dates_list = []
applicants_count_list = []


try: 
    for page in range(2,total_pages+1):
        time.sleep(2)

        #identify and get entire jobs block 
        jobs_block = driver.find_element(By.CLASS_NAME,'jobs-search-results-list')
        
        #get all job elements
        jobs_list= jobs_block.find_elements(By.CLASS_NAME, 'job-card-list__entity-lockup')
        
        for i in jobs_list:
            driver.execute_script("arguments[0].scrollIntoView();", i)
            time.sleep(2)
        #print(jobs_list)  
            jobs_block = driver.find_element(By.CLASS_NAME,'jobs-search-results-list')
            jobs_list= jobs_block.find_elements(By.CLASS_NAME, 'job-card-list__entity-lockup')
        
        jobs_block = driver.find_element(By.CLASS_NAME,'jobs-search-results-list')
        jobs_list= jobs_block.find_elements(By.CLASS_NAME, 'job-card-list__entity-lockup')
        
        
        #print(jobs_block.find_elements(By.XPATH,all_links_xpath))
        
        for job in jobs_list:
            all_links = job.find_elements(By.TAG_NAME,'a')
            #print(all_links)
            for a in all_links:
                if str(a.get_attribute('href')).startswith("https://www.linkedin.com/jobs/view") and a.get_attribute('href') not in links: 
                    links.append(a.get_attribute('href'))
                else:
                    pass
            # scroll down for each job element
            driver.execute_script("arguments[0].scrollIntoView();", job)
            driver.implicitly_wait(10)
        
        curr_time = time.strftime("%H:%M:%S", time.localtime())
        print(f'Collecting the links in the page: {page-1}',"Current Time is :", curr_time)
        
        # time.sleep(20)
        if page==8:
            driver.refresh()
            time.sleep(20)
        # go to next page:
        driver.find_element(By.XPATH,f"//button[@aria-label='Page {page}']").click()
        time.sleep(3)
        
except:
    pass


print('Found ' + str(len(links)) + ' links for job offers')
driver.close()

Starting_Time:  19:11:22
482 results
20
Links are being collected now.
Found 340 links for job offers


In [5]:
pd.DataFrame({'Links':links}).to_csv(r'D:\scrapped_jobs\Oct22_LinkedinExtractedLinks.csv',index=False)
links = pd.read_csv(r'D:\scrapped_jobs\Oct22_LinkedinExtractedLinks.csv').Links.to_list()

In [7]:
#form csv 
import datetime
# for timezone()
import pytz
job_titles = []
contact_person = []
contact_team = []
company_name =[]
company_link = []
location_name = []
employmemt_type = []
seniority_level = []
post_date = []
applicants_count = []
description = []
job_links = []

wait_point = 1

In [8]:
print(len(job_titles),len(contact_person),len(contact_team),len(company_name),len(company_link),len(location_name),len(employmemt_type),len(seniority_level),len(post_date),len(applicants_count),len(description),len(job_links))
print(wait_point)
print(links[28])
print(job_titles[27])

0 0 0 0 0 0 0 0 0 0 0 0


In [11]:
#passing the links and get the details of each job and taking the contacts
start_time = time.strftime("%H:%M:%S", time.localtime())
print('Start time:', start_time)
for link in links:
    
    if link not in job_links:
        op = webdriver.ChromeOptions()
        op.add_argument('headless')
        driver = webdriver.Chrome(options=op)
        # driver = webdriver.Chrome()
        #link = 'https://www.linkedin.com/jobs/view/3686754061/?eBP=CwEAAAGKxxnSTkzu5AJfFnurpPVatjpllHHB32r-ddM530uibLmc_RchDB6r_7r49B-A0SpNg1NqRKoXbXEXVTGdl9B_cg4jq9HmhNYOCEZejWlxlEazXsQqFxha6p7WBo5JbgJKRfuRk8OOTQJZbUPyIAom2baGvyjrnAl3p_IUF2U3dxIVHgqZzSJScdQWyzQmMoGW1XtS6AOEybaJeroFuLf74lX0ifI4te5r7poxBhxyjYI6rad0N4zOe-brnZd2v_emaBfnbFvfueOH4KhV2DaJk8vNNg5GK-HeXhMaSo0wFiKhl-wEPjfKzts9G-Tho9DmrD1EN1BGu7LmfQJV-97Sw3Fx98jsk7Xmedsk0xWIjI5uGs5zl97sUVSwMj1IafYhWwY&refId=ceCqJZR9Mvb7jxY3ND9zqA%3D%3D&trackingId=yrX7jnEhAflhLz6e%2BXkACQ%3D%3D&trk=flagship3_search_srp_jobs'
        driver.get(link)
        driver.maximize_window()
        time.sleep(2)
        # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # time.sleep(1)
        driver.find_element(By.CLASS_NAME,"show-more-less-html__button").click()
        
        top_card = driver.find_element(By.CLASS_NAME,'top-card-layout__card')
        #print('job_title-->',top_card.find_element(By.TAG_NAME,'h1').text)
        job_titles.append(top_card.find_element(By.TAG_NAME,'h1').text)
        
        try:
            person = driver.find_element(By.CSS_SELECTOR,"div.message-the-recruiter>div>div>h3").text
            team = driver.find_element(By.CSS_SELECTOR,"div.message-the-recruiter>div>div>h4").text
            print(person,team)
            contact_person.append(person)
            contact_team.append(team)
        except:
            contact_person.append('')
            contact_team.append('')
        
        #print('company_name-->',top_card.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/section[2]/div/div[1]/div/h4/div[1]/span[1]/a').text)
        company_name.append(top_card.find_element(By.CSS_SELECTOR,'div.top-card-layout__entity-info>h4>div>span').text)
        
        #print('company_link-->',top_card.find_element(By.TAG_NAME,'a').get_attribute('href'))
        company_link.append(top_card.find_element(By.TAG_NAME,'a').get_attribute('href'))
        
        #print('location-->',top_card.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/section[2]/div/div[1]/div/h4/div[1]/span[2]').text)
        location_name.append(top_card.find_element(By.CSS_SELECTOR,'div.top-card-layout__entity-info>h4>div>span.topcard__flavor--bullet').text)
        
        #print('post_dates-->',top_card.find_element(By.CLASS_NAME,'posted-time-ago__text').text)
        post_date.append(top_card.find_element(By.CLASS_NAME,'posted-time-ago__text').text)
        
        #print('applicants_count-->',top_card.find_element(By.CLASS_NAME,'num-applicants__caption').text)
        applicants_count.append(top_card.find_element(By.CLASS_NAME,'num-applicants__caption').text)
        
        job_criteria_text_list = [i.text for i in driver.find_elements(By.CSS_SELECTOR,"span.description__job-criteria-text")]
        
        #employment type
        if len(job_criteria_text_list) >= 2:
            employmemt_type.append(job_criteria_text_list[1])
        else:
            employmemt_type.append('None')
        #print('seniority_level-->',top_card.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div/section[1]/div/ul/li[1]/span').text)
        seniority_level.append(job_criteria_text_list[0])
        
        #print(driver.find_element(By.CLASS_NAME,'description__text').text)
        description.append(driver.find_element(By.CLASS_NAME,'description__text').text)
        job_links.append(link)
        
        driver.close()

        wait_point += 1
        
        if wait_point % 15 == 0:
            
            # using now() to get current time
            current_time = time.strftime("%H:%M:%S", time.localtime())
            print(f"{wait_point} links completed"," The current time in india is :", current_time)
            
            time.sleep(10)
            
    else:
        
        pass
        
df = pd.DataFrame({
    'job_title':job_titles,'contact_person':contact_person,'company_name':company_name,'company_link':company_link,
    'location_name':location_name,'post_dates':post_date,
    'applicants_count':applicants_count,'employment_type':employmemt_type,
    'seniority_level':seniority_level,'description':description,'Job_links':job_links
})

df.to_csv(r'D:\scrapped_jobs\Oct22_data_engineer_intern_linkedin_jobs.csv',index=False)
df.head()


Start time: 19:35:59
15 links completed  The current time in india is : 19:37:14
30 links completed  The current time in india is : 19:39:33
Lalit Shukla Innovator | Quantum Computing | Deep neural networks
45 links completed  The current time in india is : 19:41:58
Kamini Kumari Singh Talent Acquisition Executive I - US IT Recruiter at Codvo.ai Hiring for US IT Roles..C2C/W2/USC/GC/H4 and for India AWS Devops and Data Scientist
Prashant Kumar Technical Recruiter at IntraEdge
Muhammed Irfan Ali C T Director at Webcoffee Innovations | Engineering, Design, Development
60 links completed  The current time in india is : 19:44:22
75 links completed  The current time in india is : 19:46:43
90 links completed  The current time in india is : 19:49:04
Priyanka Meena Talent Hire Specialist, PeopleSoft, D365,DevOps, SAP, ServiceNow,Big Data, Python,Java, Cypress QA etc...
105 links completed  The current time in india is : 19:51:27
Garima Shukla Human resource
120 links completed  The current tim

,job_title,contact_person,company_name,company_link,location_name,post_dates,applicants_count,employment_type,seniority_level,description,Job_links
0,Hadoop Cluster Developer,,Infosys,https://in.linkedin.com/company/infosys?trk=pu...,"Trivandrum, Kerala, India",2 days ago,52 applicants,Full-time,Entry level,Experience in Hadoop cluster management – Incl...,https://www.linkedin.com/jobs/view/3730472762/...
1,Microsoft Engineer,,NTT Ltd.,https://uk.linkedin.com/company/global-ntt?trk...,"Hyderabad, Telangana, India",2 days ago,125 applicants,Full-time,Entry level,Want to be a part of our team?\n\nAbout us:\n\...,https://www.linkedin.com/jobs/view/3730211764/...
2,Clinical Data Science Programmer,,ICON plc,https://ie.linkedin.com/company/icon-plc-2?trk...,"Bengaluru, Karnataka, India",5 days ago,171 applicants,Full-time,Entry level,ICON plc is a world-leading healthcare intelli...,https://www.linkedin.com/jobs/view/3741898600/...
3,Outsystems Development,,UST,https://www.linkedin.com/company/ustglobal?trk...,"Kochi, Kerala, India",6 days ago,25 applicants,Full-time,Entry level,Job Description\n\nRole Proficiency:\n\nAct cr...,https://www.linkedin.com/jobs/view/3741790299/...
4,FinOps Data Analytics Engineer,,WPP,https://uk.linkedin.com/company/wpp?trk=public...,"Chennai, Tamil Nadu, India",3 days ago,124 applicants,Full-time,Entry level,WPP is the creative transformation company. We...,https://www.linkedin.com/jobs/view/3728816636/...


In [10]:
df = pd.DataFrame({
    'job_title':job_titles,'contact_person':contact_person,'company_name':company_name,'company_link':company_link,
    'location_name':location_name,'post_dates':post_date,
    'applicants_count':applicants_count,'employment_type':employmemt_type,
    'seniority_level':seniority_level,'description':description,'Job_links':job_links
})

df.to_csv(r'D:\scrapped_jobs\Oct22_data_engineer_intern_intern_22ndOCT_22_linkedin_jobs.csv',index=False)
df.head()


,job_title,contact_person,company_name,company_link,location_name,post_dates,applicants_count,employment_type,seniority_level,description,Job_links
0,Analyst Data Science Analytics,,ClickJobs.io,https://uk.linkedin.com/company/clickjobsio?tr...,"Bengaluru, Karnataka, India",5 hours ago,103 applicants,Full-time,Entry level,Grow your career with a growing organization\n...,https://www.linkedin.com/jobs/view/3738351411/...
1,Data Analyst (Mutual Series),,Franklin Templeton India,https://in.linkedin.com/company/franklintemple...,"Mumbai, Maharashtra, India",2 days ago,Over 200 applicants,Full-time,Entry level,"At Franklin Templeton, everything we do is foc...",https://www.linkedin.com/jobs/view/3737214595/...
2,Process Developer – Data Analyst-COR021664,,Genpact,https://www.linkedin.com/company/genpact?trk=p...,"Noida, Uttar Pradesh, India",4 days ago,Over 200 applicants,Full-time,Entry level,"With a startup spirit and 90,000+ curious and ...",https://www.linkedin.com/jobs/view/3623351494/...
3,Forecasting Analyst,,NewAge Products Inc.,https://ca.linkedin.com/company/newage-product...,India,1 day ago,Over 200 applicants,Full-time,Entry level,"NewAge Products Inc. (NAP), headquartered in N...",https://www.linkedin.com/jobs/view/3737546187/...
4,Internal Audit - Data Strategy - Analyst - Hyd...,,Goldman Sachs,https://www.linkedin.com/company/goldman-sachs...,"Hyderabad, Telangana, India",4 hours ago,Over 200 applicants,Full-time,Entry level,Job Description\n\nWhat We Do\n\nInternal Audi...,https://www.linkedin.com/jobs/view/3648716404/...


In [32]:
for link in ["https://www.linkedin.com/jobs/view/3726846194/?eBP=CwEAAAGK6jL7xhZdwcUskrNit2JWpIJ2Mmq2mCZO0epVgihUEuR33E6Q6p6HXoSsBtwKTUbcBtZ9HqJx-Aij3gmIwhVkGDdAXz_4yYb8Yio9_dXBA72rXkSzDTC94l4_RSS2tazwUKOXrOK-5SifYc1T4lpolWpJWpHO1KwpN8o6y_PhJLUvAbxj8v-XJyEt7nPn9uibHMq3TjNXs0y4ZPGosd3Maj2UuQDCsV23fX0bDz4UU_61qeB26W0QRHRuZBtch4SoScJXVBF22WE98SltBZWyLf4GTd2Z6OaBlYMdOIkikRuSt1ChwbPmNU9FX7VDW0g3onMMzxsZwgnNLitvPd0nmX7u5NNHHw&refId=SVYUdRCkQgQn%2FNOOQhBdjg%3D%3D&trackingId=FIbt9FM3QSD8HpVkdRv74g%3D%3D&trk=flagship3_search_srp_jobs",
             'https://www.linkedin.com/jobs/view/3704252032/?eBP=CwEAAAGK6jL7xoWBvvbrxB-4IRsm2dnixY_DmZ-',
             "https://www.linkedin.com/jobs/view/3726846194/?eBP=CwEAAAGK6jL7xhZdwcUskrNit2JWpIJ2Mmq2mCZO0epVgihUEuR33E6Q6p6HXoSsBtwKTUbcBtZ9HqJx-Aij3gmIwhVkGDdAXz_4yYb8Yio9_dXBA72rXkSzDTC94l4_RSS2tazwUKOXrOK-5SifYc1T4lpolWpJWpHO1KwpN8o6y_PhJLUvAbxj8v-XJyEt7nPn9uibHMq3TjNXs0y4ZPGosd3Maj2UuQDCsV23fX0bDz4UU_61qeB26W0QRHRuZBtch4SoScJXVBF22WE98SltBZWyLf4GTd2Z6OaBlYMdOIkikRuSt1ChwbPmNU9FX7VDW0g3onMMzxsZwgnNLitvPd0nmX7u5NNHHw&refId=SVYUdRCkQgQn%2FNOOQhBdjg%3D%3D&trackingId=FIbt9FM3QSD8HpVkdRv74g%3D%3D&trk=flagship3_search_srp_jobs",
             "https://www.linkedin.com/jobs/view/3726780795/?eBP=JOB_SEARCH_ORGANIC&refId=TrSZkvmL8BzLsdZxhyQd4g%3D%3D&trackingId=ChlB%2BWl5Ntwg4HvqNMmFYA%3D%3D&trk=flagship3_search_srp_jobs"]:
    
    # op = webdriver.ChromeOptions()
    # op.add_argument('headless')
    # driver = webdriver.Chrome(options=op)
    driver = webdriver.Chrome()
    driver.get(link)
    driver.find_element(By.CLASS_NAME,"show-more-less-html__button").click()
    driver.maximize_window()
    
    try:
        person = driver.find_element(By.CSS_SELECTOR,"div.message-the-recruiter>div>div>h3").text
        team = driver.find_element(By.CSS_SELECTOR,"div.message-the-recruiter>div>div>h4").text
        print(person,team)
        contact_person.append(person)
        contact_team.append(team)
    except:
        contact_person.append('None')
        contact_team.append('None')
        

    #top_card = driver.find_element(By.CLASS_NAME,'top-card-layout__card')

    print([i.text for i in driver.find_elements(By.CSS_SELECTOR,"span.description__job-criteria-text")])
    driver.close()

['Entry level', 'Full-time', 'Engineering and Information Technology', 'Software Development']
['Entry level', 'Full-time', 'Business Development and Sales', 'IT Services and IT Consulting']
['Entry level', 'Full-time', 'Engineering and Information Technology', 'Software Development']
Deepakarasi V Recruiter at Digispoc Technology
['Full-time']


In [97]:
#merging all the datasets and getting only contact
import pandas as pd
import glob
import numpy as np
import datetime

#Student Individual 2.1
def read_file(): #reading files inside the folders
    
    csv_path = r'D:\scrapped_jobs' #

    # filenames = glob.glob(path + "\*.xlsx")
    path_filenames = glob. glob(csv_path + "/*.csv")
    path_filenames.sort()
#     print('File names:', csv_filenames)
    
    return path_filenames

AllFiles=read_file()

def file_names(path_filenames): #getting file names
    files = []
    
    for index,file in enumerate(path_filenames):
        f = file.split("D:\\scrapped_jobs\\")[1].split(".csv")[0]
        files.append(f'{f}')

    return files

files = file_names(read_file())

merged_frame = pd.DataFrame({'job_title':[], 'contact_person':[], 'company_name':[],'Job_links':[],'scrap_date':[]})

for i,j in zip(AllFiles,files):
    
    month_list = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    
    df = pd.read_csv(i) #creating dataframe by reading csv files
    
    #finding the scrap date and adding it to the dataframe
    scrap_day = j.split('_')[0]
    if scrap_day[0:3] in month_list:
        scrap_date = datetime.datetime(2023, month_list.index(scrap_day[0:3])+1, int(scrap_day[3:]))
        df['scrap_date'] = scrap_date
        columns_to_take = ['job_title', 'contact_person', 'company_name','Job_links','scrap_date']
    
        try:
            df = df.loc[:,columns_to_take]
            bool_index = df.contact_person.apply(lambda x: type(x).__name__) != 'float'
            index_list = [j for j,i in enumerate(bool_index) if i == True]
            df = df.iloc[index_list,[0,1,2,3,4]]
            merged_frame = pd.concat([merged_frame,df],ignore_index=True)
            
        except:
            pass
    
merged_frame.sort_values('scrap_date',inplace=True)
merged_frame.to_csv(r'D:\scrapped_jobs\LinkedinContactsData.csv',index=False)
